In [1]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import Settings
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.prompts import PromptTemplate
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.node_parser import SentenceSplitter,SentenceWindowNodeParser
from llama_index.core.extractors import (SummaryExtractor,TitleExtractor,KeywordExtractor)
from llama_index.core.ingestion import IngestionPipeline
from llama_index.readers.azstorage_blob import AzStorageBlobReader
from llama_index.core.extractors import BaseExtractor
from llama_index.vector_stores.postgres import PGVectorStore
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core import Document
from llama_index.core.node_parser import (SentenceSplitter,SemanticSplitterNodeParser)

In [2]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from gensim.models import LdaModel
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from llama_index.core import PromptTemplate
from gensim import corpora
from llama_index.core.query_pipeline import QueryPipeline
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shree\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shree\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
reader = AzStorageBlobReader(
    container_name='docs',
    connection_string='DefaultEndpointsProtocol=https;AccountName=shreemirrahrag;AccountKey=UlbcKqbBtcs0J+2esmr8AznpwdV1dF5XI+v13kY07sjJ2U8rHmWbtCUqLjb12lD7Bn9k17mWgqjd+AStB4xXcw==;EndpointSuffix=core.windows.net'
)

documents = reader.load_data()

In [4]:
api_key = "c09f91126e51468d88f57cb83a63ee36"
azure_endpoint = "https://chat-gpt-a1.openai.azure.com/"
api_version = "2023-03-15-preview"

In [5]:
llm = AzureOpenAI(
    model="gpt-35-turbo-16k",
    deployment_name="DanielChatGPT16k",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [6]:
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-3-small",
    deployment_name="text-embedding-3-small",
    api_key="c09f91126e51468d88f57cb83a63ee36",
    azure_endpoint="https://chat-gpt-a1.openai.azure.com/",
    api_version="2023-03-15-preview",
)

In [7]:
import nest_asyncio
nest_asyncio.apply()

In [8]:
Settings.llm = llm
Settings.embed_model = embed_model

In [9]:
transformations = [TitleExtractor(nodes=5),SummaryExtractor(summaries=["prev", "self"])]

In [10]:
pipeline = IngestionPipeline(transformations=transformations)
nodes_parsed = pipeline.run(documents=documents)

100%|██████████| 104/104 [00:33<00:00,  3.15it/s]


In [11]:
data = {
    "Agreement": ["agreement", "license agreement", "database access and license agreement"],
    "Parties": ["parties", "licensee", "licensor", "CoreLogic Solutions, LLC", "Bright MLS, Inc.", "ClearCapital.com, Inc."],
    "Effective Date": ["effective date", "July 2017", "August 4, 2021","August 4, 2021,"],
    "Content": ["content", "approved content", "field list", "Bright Content", "BRIGHT IP", "Bright Service", "Bright Database"],
    "Ownership": ["ownership", "intellectual property", "Bright IP", "Licensor's Rights", "proprietary rights", "copyrights"],
    "Royalty Fees": ["royalty fees", "license fees"],
    "Terms and Conditions": ["standard terms and conditions", "additional restrictions", "limitations"],
    "Notices": ["copyright notice", "proprietary rights notices"],
    "Reports": ["property level reports", "appraisal review reports", "property condition reports", "fraud reports"],
    "Data": ["MLS data", "public record data", "raw data", "data services"],
    "Valuation": ["valuation models", "automated valuation models (AVMs)"],
    "Renewal": ["term", "autorenewal", "renewal provisions"],
    "Termination": ["termination provisions", "terminate for convenience"],
    "Notice": ["notice", "providing notice", "requirements for providing notice"],
    "Address": ["address", "notice address", "address for notice"],
    "Database Access": ["database access", "BRIGHT database access"],
    "Field List": ["approved content field list"],
    "Listing": ["listing reports", "sample listing reports"],
    "Intellectual Property": ["intellectual property", "copyrights", "proprietary rights"],
    "Sublicense Agreements": ["sublicense agreements", "required terms in sublicense agreements"],
    "Dates of Singature": ["7/19/2017","12/22/17", "5/13/2019","8/4/2021","8/5/2021"]
}


In [12]:
class CustomExtractor(BaseExtractor):
    async def aextract(self, nodes):
        metadata_list = []
        for node in nodes:
            metadata = {}
            metadata["custom"] = node.metadata["document_title"] + "\n"
            metadata["keywords"] = []
            for key in data:
                for keyword in data[key]:
                    if keyword.lower() in node.text.lower():
                        metadata["keywords"].append(key+' : '+keyword)
            metadata_list.append(metadata)
        return metadata_list

In [13]:
extractor = CustomExtractor()
keyword=extractor.extract(nodes_parsed)

In [14]:
for node, keywords in zip(nodes_parsed, keyword):
    node.metadata['keywords']= keywords

In [15]:
nodes_parsed[0].metadata

{'page_label': '1',
 'file_name': 'BPP_Bright final agreement_SPB signature_5.13.19 (15).pdf',
 'document_title': 'MLS Agreement Definitions and Terms, MLS License and Data Usage Agreement, MLS License Agreement and Data Access Terms, Agreement Terms for Homesnap Pro Suite, Premium Products, and MLS Fees, Agreement Terms and Termination for NBP and MLS. \n\nComprehensive Title: MLS Agreement and License Terms, Data Usage, and Termination',
 'section_summary': 'The key topics of the section are the MLS Agreement between the Multiple Listing Service (MLS) and National Broker Portal LLC (NBP), and the definitions of various terms used in the agreement. The entities involved in the agreement are the MLS, NBP, and the Participants (brokers) who provide data to the MLS.',
 'keywords': {'custom': 'MLS Agreement Definitions and Terms, MLS License and Data Usage Agreement, MLS License Agreement and Data Access Terms, Agreement Terms for Homesnap Pro Suite, Premium Products, and MLS Fees, Agreem

In [16]:
from typing import List, Optional
import itertools
import pandas as pd
from pydantic import BaseModel, Field, field_validator
from kor import extract_from_documents, from_pydantic, create_extraction_chain
from langchain_core.documents import Document
from langchain_openai import AzureChatOpenAI

In [17]:
llm_langchain = AzureChatOpenAI(api_key="c09f91126e51468d88f57cb83a63ee36",api_version="2023-03-15-preview",azure_deployment="DanielChatGPT16k",azure_endpoint="https://chat-gpt-a1.openai.azure.com/",temperature=0,max_tokens=2000, model_kwargs = {'frequency_penalty':0,'presence_penalty':0,'top_p':1.0})


In [18]:
class EntityExtraction(BaseModel):
    documents_name: Optional[str] = Field(description="The name of the agreement; title",)
    parties: Optional[str] = Field(description="The company names that are involved in the making of the agreement")    
    address: Optional[str] = Field(description="The address that corresponds to each party, in the agreement",)
    effective_date: Optional[str] = Field(description="The date when the agreement came into effect.",)
    signature_date: Optional[str] = Field(description="Date(s) on which the agreement was signed by each party. Such dates are in the format 'MM/DD/YYYY'",)
    term: Optional[str] = Field(description="The term is usually mentioned its corresponding section. In-detail duration and other specification of the agreement")


In [19]:
schema, extraction_validator = from_pydantic(
    EntityExtraction,
    description="Extract information about the overall agreement based on the text provided.",
    examples=[
        (
            """BRIGHT DATABASE ACCESS AND LICENSE AGREEMENT
                ORDER FORM
            This BRIGHT Database Access and License Agreement is made as of July 2017, (the “Effective Date™), between BRIGHT MLS, INC., a
            Delaware corporation, with offices at 660 American Avenue, King of Prussia, PA, 19406 ("BRIGHT" or “Licensor”) and the licensee listed below (“Licensee” or “CoreLogic™).
            Licensee: CoreLogic Solutions, LLC Contact:
            Address: 40 Pacifica, Suite 900 Phone:
            Trvine, CA 92618
            Email: """,
            {"documents_name": "BRIGHT DATABASE ACCESS AND LICENSE AGREEMENT",
             "parties": ["BRIGHT MLS, INC. “Licensor”","CoreLogic Solutions, LLC “Licensee”"],
             "address":["660 American Avenue, King of Prussia, PA, 19406","40 Pacifica, Suite 900 Trvine, CA 92618"],
             "effective_date":"July 2027",
             "signature_date":None,
             "term":None
             },
        ),
        (
            """CoreLogic Solutions, LLC: BRIGHT MLS, INC. Docusigned by
            (. ot By: Brin Dncllr
            eneredrManager, Real Estate Solutions Title: COO Title:
            Date: 7/19/2017 Date: 7/19/2017 """,
            {
            "documents_name": None,
             "parties": "CoreLogic Solutions, LLC",
             "address":None,
             "effective_date":None,
                "signature_date":["7/19/2017","7/19/2017"],
                "term":None
             },
        ),
        (
            """Term and Termination.
51 Term. The initial term of the Agreement shall commence on the Effective Date and shall remain in effect for twenty- four (24) months from the Effective Date (the “Initial Term™), unless terminated carlier as provided in the Agreement. The Agreement shall be automatically renewed after the Initial Term for twelve (12) month terms (each, a “Renewal Term™) thereafter unless either party gives written notice of termination at least sixty (60) days prior to expiration of the Initial Term or the current Renewal Term as provided in the Agreement. (The Initial Term, as extended by the Renewal Terms, collectively constitutes the “Term™.)
5.2 Termination for Breach. The Agreement shall terminate, for material breach, if Licensee breaches any provision of the BRIGHT Policies or the scope of the license granted herein and fails to cure the breach within fifteen (15) days of BRIGHT’s written notice of the breach. Additionally, cither party may terminate this Agreement if the other party materially breaches any other provision of this Agreement and fails to cure the breach within thirty (30) days of the non-breaching party's written notice of the breach. """,
            {
            "documents_name": None,
             "parties": "CoreLogic Solutions, LLC",
             "address":None,
             "effective_date":None,
                "signature_date":["7/19/2017","7/19/2017"],
                "term":"The initial term of the Agreement shall commence on the Effective Date and shall remain in effect for twenty- four (24) months from the Effective Date (the “Initial Term™), unless terminated carlier as provided in the Agreement. The Agreement shall be automatically renewed after the Initial Term for twelve (12) month terms (each, a “Renewal Term™) thereafter unless either party gives written notice of termination at least sixty (60) days prior to expiration of the Initial Term or the current Renewal Term as provided in the Agreement. (The Initial Term, as extended by the Renewal Terms, collectively constitutes the “Term™.)"
             },
        ),
        

    ],
    many=True,
)

In [20]:
chain = create_extraction_chain(
    llm_langchain,
    schema,
    encoder_or_encoder_class="json",
    validator=extraction_validator,
    input_formatter="triple_quotes",
)

c:\Users\shree\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [21]:
from langchain_community.callbacks import get_openai_callback

In [22]:
docs=[]
for node in nodes_parsed:
    doc=Document(page_content=node.text)
    docs.append(doc)

In [23]:
with get_openai_callback() as cb:
    document_extraction_results = await extract_from_documents(
        chain, docs, max_concurrency=5, use_uid=False, return_exceptions=True
    )

c:\Users\shree\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.arun` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use ainvoke instead.
  warn_deprecated(


In [24]:
len(nodes_parsed)

104

In [53]:
len(document_extraction_results)

104

In [80]:
document_extraction_results[0]

{'uid': '0',
 'source_uid': '0',
 'data': {'entityextraction': [{'documents_name': 'Broker Public Portal MLS Agreement',
    'parties': ['MLS', 'National Broker Portal LLC'],
    'address': None,
    'effective_date': None,
    'signature_date': None,
    'term': None}]},
 'raw': '<json>{"entityextraction": [{"documents_name": "Broker Public Portal MLS Agreement", "parties": ["MLS", "National Broker Portal LLC"], "address": null, "effective_date": null, "signature_date": null, "term": null}]}</json>',
 'validated_data': [],
 'errors': [1 validation error for EntityExtraction
  parties
    Input should be a valid string [type=string_type, input_value=['MLS', 'National Broker Portal LLC'], input_type=list]
      For further information visit https://errors.pydantic.dev/2.8/v/string_type]}

In [26]:
for node, entity in zip(nodes_parsed, document_extraction_results):
    node.metadata['entity']= entity['data']

Sample metadata for reference

In [81]:
nodes_parsed[0].metadata

{'page_label': '1',
 'file_name': 'BPP_Bright final agreement_SPB signature_5.13.19 (15).pdf',
 'document_title': 'MLS Agreement Definitions and Terms, MLS License and Data Usage Agreement, MLS License Agreement and Data Access Terms, Agreement Terms for Homesnap Pro Suite, Premium Products, and MLS Fees, Agreement Terms and Termination for NBP and MLS. \n\nComprehensive Title: MLS Agreement and License Terms, Data Usage, and Termination',
 'section_summary': 'The key topics of the section are the MLS Agreement between the Multiple Listing Service (MLS) and National Broker Portal LLC (NBP), and the definitions of various terms used in the agreement. The entities involved in the agreement are the MLS, NBP, and the Participants (brokers) who provide data to the MLS.',
 'keywords': {'custom': 'MLS Agreement Definitions and Terms, MLS License and Data Usage Agreement, MLS License Agreement and Data Access Terms, Agreement Terms for Homesnap Pro Suite, Premium Products, and MLS Fees, Agreem

In [27]:
def topic_tf_idf(text):
    sentences = [sentence.strip() for sentence in text.split(".")]
    df = pd.DataFrame(sentences, columns=["text"])
    vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words="english")
    tfidf = vectorizer.fit_transform(df["text"])
    lda = LatentDirichletAllocation(n_components=5, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tfidf)
    topic_words = []
    for topic in lda.components_:
        topic_words.append([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-10 - 1:-1]])
    topic_list=[topic for topic in topic_words]
    return topic_list

def retreive(template_str,string_lda,num_topics=2):
    num_topics=str(num_topics)
    prompt_tmpl = PromptTemplate(template_str)
    p = QueryPipeline(chain=[prompt_tmpl, llm], verbose=True)
    output = p.run(string_lda=string_lda,num_topics=num_topics)
    return str(output)

def topics_from_text(llm, text):
    list_of_topicwords = topic_tf_idf(text)
    string_lda = ""
    for list in list_of_topicwords:
        string_lda += str(list) + "\n"
    template_string = """Describe the topic of each of the {num_topics} 
        double-quote delimited lists in a simple sentence. The lists are the result of an 
        algorithm for topic discovery.
        Do not provide an introduction or a conclusion, only describe the 
        topics. Do not mention the word "topic" when describing the topics.
        Use the following template for the response.
        
        Topic: <<<(sentence describing the topic)>>>

        Topic: <<<(sentence describing the topic)>>>
        ...

        n: <<<(sentence describing the topic)>>>



        Lists: {string_lda} """

    # LLM call
    #num_topics=str(num_topics)
    response=retreive(template_string, string_lda)
    return response

In [28]:
class CustomExtractor(BaseExtractor):
    async def aextract(self, nodes):
        metadata_list = []
        for node in nodes:
            metadata = {}
            try:
                metadata["TF_IDF"] = topics_from_text(llm,node.text)
            except ValueError:
                continue
            metadata_list.append(metadata)
        return metadata_list

In [29]:
extractor = CustomExtractor()
tf_idf=extractor.extract(nodes_parsed)

> Running module 62748b34-e260-45ab-a043-e60b6354e608 with input: 
string_lda: ['suite', 'homesnap', 'include', 'products', 'developed', 'party', 'pro', 'disclosure', 'confidential', 'does']
['media', 'data', 'mls', 'basic', 'licensed', 'choose', 'publisher', 'listings', 'includ...
num_topics: 2

> Running module c0bf3c1b-d36d-478a-8927-69f53afa2c8d with input: 
messages: Describe the topic of each of the 2 
        double-quote delimited lists in a simple sentence. The lists are the result of an 
        algorithm for topic discovery.
        Do not provide an introdu...

> Running module e9f41aa6-58d2-4cfd-8791-06aff5ac1f51 with input: 
string_lda: ['information', 'roster', 'agreement', 'nbp', 'basic', 'display', 'hs', 'shall', 'fields', 'subscriber']
['listings', 'license', 'basic', 'email', 'property', 'participant', 'subscribers', 'sold', 'co...
num_topics: 2

> Running module a822752d-24d9-426f-85e1-d050ae5bbd17 with input: 
messages: Describe the topic of each of the 2 
        

In [30]:
for node, topics in zip(nodes_parsed, tf_idf):
    node.metadata['lda_topcis']= topics

In [32]:
index=VectorStoreIndex(nodes_parsed)

In [33]:
context_str="""You're an assistant who fetches answers from the documents. Add all the necessary details."""

In [34]:
template = """
You are a knowledgeable and precise assistant specialized in question-answering tasks, 
particularly from documented sources. 
Your goal is to provide accurate, concise, and contextually relevant answers based on the given information.

You are provided with 4 documents, namely: BPP_Bright final agreement_SPB signature_5.13.19 (15), Bright CoreLogic Infonet Agreement (Executed), CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728 and Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4).
You have to answer each question for each of the four documents:
Use this format for generating response:
'''
Document 1: BPP_Bright final agreement_SPB signature_5.13.19 (15)
<your response pertaining to BPP_Bright final agreement_SPB signature_5.13.19 (15), based on the metadata where file_name:  BPP_Bright final agreement_SPB signature_5.13.19 (15).pdf>

Document 2: Bright CoreLogic Infonet Agreement (Executed)
<your response pertaining to Bright CoreLogic Infonet Agreement (Executed), based on the metadata where file_name:  Bright CoreLogic Infonet Agreement (Executed).pdf>

Document 3: CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728
<your response pertaining to BPP_Bright final agreement_SPB signature_5.13.19 (15), based on the metadata where file_name:  CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728.pdf>

Document 4: Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4)
<your response pertaining to BPP_Bright final agreement_SPB signature_5.13.19 (15), based on the metadata where file_name:  Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4).pdf>
'''

Instructions:

Answer the following question using the provided context. Make sure your answer is concise, accurate, and contextually relevant.
Terms and Policies: When you're asked questions like "What's the term of the agreement?", you should first check the section number that corresponds to the 'term' and provide the appropriate answer. This also applies to other terminologies such as 'policies','confidentiality', 'trademarks', 'intellectual property rights' and so on. You must refer the section number which corresponds to the terms, and provide the relevant, detailed answer mentioned in the section
Parties Involved: when referring to the parties involved, find the 'specific names' of the parties along with their role and not just their role. For example, don't refer to parties by just 'Licensee' use '<company name> (Licensee)'
Parties involved in the agreement: Keep in mind that the parties referred to the agreement are actually the 'Companies' involved in the particular agreement. Hence, while referring to 'parties', generate the company names that play a role in the agreement. Never mention solely the role, such as 'Licensee' or 'Licensor' these are not the companies' names. Always mention the "Companies' name"
Dates: When it comes to retrieving information pertaining to dates in the document, look for the event/term associated with each date. For example, effective date is associated with the keyword 'effective date' or any of its synonyms and you must find the date corresponding to that. Similarly, it's important to find dates of signature in the document. They're usually(not always) denoted in the format __/__/____
Dates of Signatue: Any date of the format: __/__/____ Eg: 8/4/2021, or 8/5/2021 are dates on which the documents were signed (dates of signature)
Address: An address is of the following format: <Number> <Street name> <Area name> <Pin code> for example: 660 American Avenue, King of Prussia, PA, 19406 
Comprehension and Accuracy: Carefully read and comprehend the provided context from the documents to ensure accuracy in your response.
Truthfulness: If the context does not provide enough information to answer the question, clearly state, "I don't know."
Contextual Relevance: Ensure your answer is well-supported by the retrieved context and does not include any information beyond what is provided.

If asked any question like: "Who are the parties to the agreement?": answer the question for "Who are the companies to the agreement?"

Remember if no context is provided please say you don't know the answer
Here is the question and context for you to work with:

\nQuestion: {question} \nContext: {context} \nAnswer:"""

In [35]:
prompt_tmpl = PromptTemplate(
    template=template,
    template_var_mappings={"query_str": "question", "context_str": "context"},
)

In [31]:
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [36]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5
)

rerank = SentenceTransformerRerank(
    top_n = 5, 
     model = "BAAI/bge-reranker-base"
 )
postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

response_synthesizer = get_response_synthesizer()

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,node_postprocessors=[rerank,postproc]
)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template":prompt_tmpl}
)

c:\Users\shree\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [77]:
question="Who are the parties to the agreement?"

In [78]:
streaming_response = query_engine.query(question)

In [89]:
print(streaming_response)


Document 1: BPP_Bright final agreement_SPB signature_5.13.19 (15)
The parties to the agreement mentioned in this document are NATIONAL BROKER PORTAL, LLC, Homesnap, and Bright MLS Inc.

Document 2: Bright CoreLogic Infonet Agreement (Executed)
I don't know.

Document 3: CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728
I don't know.

Document 4: Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4)
The parties to the agreement mentioned in this document are ShowingTime™ (the provider) and Bright MLS (the client).



In [40]:
question1="What is the name of the agreement?"

In [41]:
streaming_response = query_engine.query(question1)

In [42]:
print(streaming_response)

Document 1: BPP_Bright final agreement_SPB signature_5.13.19 (15)
Document 2: Bright CoreLogic Infonet Agreement (Executed)
Document 3: CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728
Document 4: Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4)

The name of the agreement is not mentioned in the provided context.


In [82]:
question1="What are the parties referred to in the agreement?"

In [83]:
streaming_response = query_engine.query(question1)

In [86]:
print(streaming_response)


Document 1: BPP_Bright final agreement_SPB signature_5.13.19 (15)
The parties referred to in the agreement are NATIONAL BROKER PORTAL, LLC, Homesnap, and Bright MLS Inc.

Document 2: Bright CoreLogic Infonet Agreement (Executed)
I don't know.

Document 3: CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728
I don't know.

Document 4: Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4)
The parties referred to in the agreement are ShowingTime™ (the provider) and Bright MLS (the client).



In [46]:
question2="What is the effective date of the agreement?"

In [47]:
streaming_response = query_engine.query(question2)

In [48]:
print(streaming_response)

Document 1: BPP_Bright final agreement_SPB signature_5.13.19 (15)
The effective date of the agreement is July 2017.

Document 2: Bright CoreLogic Infonet Agreement (Executed)
I don't know.

Document 3: CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728
I don't know.

Document 4: Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4)
I don't know.


In [49]:
question2="What's the date of last signature"

In [50]:
streaming_response = query_engine.query(question2)

In [51]:
print(streaming_response)

Document 1: BPP_Bright final agreement_SPB signature_5.13.19 (15)
Date of last signature: 7/19/2017

Document 2: Bright CoreLogic Infonet Agreement (Executed)
Date of last signature: 5/13/2019

Document 3: CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728
Date of last signature: I don't know.

Document 4: Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4)
Date of last signature: I don't know.


In [52]:
question2="What is the term of the agreement?"

In [53]:
streaming_response = query_engine.query(question2)

In [54]:
print(streaming_response)

Document 1: BPP_Bright final agreement_SPB signature_5.13.19 (15)
The term of the agreement is defined as "the period during which the terms and conditions of this Agreement are in full force and effect, as defined in Section 6.1."

Document 2: Bright CoreLogic Infonet Agreement (Executed)
I don't know.

Document 3: CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728
I don't know.

Document 4: Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4)
I don't know.


In [55]:
question2="Will the term of the agreement be auto-renewed?"

In [56]:
streaming_response = query_engine.query(question2)

In [57]:
print(streaming_response)

Document 1: BPP_Bright final agreement_SPB signature_5.13.19 (15)
The term of the agreement will be auto-renewed for successive one-year terms, unless either party provides a written ninety (90) day notice of termination to the other party. This information is mentioned in Section 6.1 of the document.

Document 2: Bright CoreLogic Infonet Agreement (Executed)
I don't know.

Document 3: CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728
I don't know.

Document 4: Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4)
I don't know.


In [58]:
question2="What are the renewal provisions of the agreement?"

In [59]:
streaming_response = query_engine.query(question2)

In [60]:
print(streaming_response)

Document 1: BPP_Bright final agreement_SPB signature_5.13.19 (15)
I don't know.

Document 2: Bright CoreLogic Infonet Agreement (Executed)
I don't know.

Document 3: CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728
I don't know.

Document 4: Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4)
The renewal provisions of the agreement state that after the initial term of two years, the agreement will automatically renew for successive one-year terms, unless either party provides a written notice of termination at least sixty (60) days prior to the expiration of the initial term or the current renewal term.


In [61]:
question2="How far in advance must a party provide notice of its intention not to renew the agreement?"

In [62]:
streaming_response = query_engine.query(question2)

In [63]:
print(streaming_response)

Document 1: BPP_Bright final agreement_SPB signature_5.13.19 (15)
I don't know.

Document 2: Bright CoreLogic Infonet Agreement (Executed)
I don't know.

Document 3: CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728
I don't know.

Document 4: Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4)
I don't know.


In [64]:
question2="What are the requirements for providing notice?"

In [65]:
streaming_response = query_engine.query(question2)

In [66]:
print(streaming_response)

The requirements for providing notice are not mentioned in the provided context.


In [67]:
print(streaming_response)

The requirements for providing notice are not mentioned in the provided context.


In [68]:
question2="What address does each party have listed for notice?"

In [69]:
streaming_response = query_engine.query(question2)

In [70]:
print(streaming_response)

Document 1: BPP_Bright final agreement_SPB signature_5.13.19 (15)
The address listed for notice for each party is as follows:
- National Broker Portal, LLC: 7200 Wisconsin Avenue, Suite 200, Bethesda, MD 20814
- Homesnap: 7200 Wisconsin Avenue, Suite 200, Bethesda, MD 20814
- Bright MLS Inc.: 9707 Key West Ave, Suite 200, Rockville MD 20850

Document 2: Bright CoreLogic Infonet Agreement (Executed)
I don't know.

Document 3: CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728
I don't know.

Document 4: Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4)
I don't know.


In [71]:
question2="What are the termination provisions?"

In [72]:
streaming_response = query_engine.query(question2)

In [73]:
print(streaming_response)

Document 1: BPP_Bright final agreement_SPB signature_5.13.19 (15)
I don't know.

Document 2: Bright CoreLogic Infonet Agreement (Executed)
I don't know.

Document 3: CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728
I don't know.

Document 4: Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4)
The termination provisions are outlined in Section 6.2 of the document. According to this section, the agreement can be terminated under the following circumstances:
(a) At the end of any term, provided that either party provides at least 30 days' notice prior to the end of the term.
(b) MLS may terminate the agreement with 180 days' written notice to NBP upon the dissolution of NBP or the exit of Broker Public Portal LLC from NBP.
(c) 10 days after either party's notice to the other that the other has breached the agreement, provided the breach remains uncured.
(d) Immediately upon any party's notice to the other that the other has breached the agreement, prov

In [74]:
question2="Can a party terminate for convenience?"

In [75]:
streaming_response = query_engine.query(question2)

In [76]:
print(streaming_response)

Document 1: BPP_Bright final agreement_SPB signature_5.13.19 (15)
I don't know.

Document 2: Bright CoreLogic Infonet Agreement (Executed)
I don't know.

Document 3: CC_Bright_Clear Capital Access and License Agreement_FINAL - 20210728
I don't know.

Document 4: Signed ShowingTime for the MLS Agreement with Bright MLS 1.1.18 (4)
I don't know.
